# [1단계] 라이브러리 설치

## 1-1. 라이브러리 설치

Colab 환경에서 필요한 라이브러리를 설치합니다. Hugging Face Hub에 접근하기 위해 `huggingface_hub`도 같이 설치합니다.

In [1]:
!pip install -q datasets transformers accelerate torch ragas huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 1-2. Hugging Face 로그인

HF_TOKEN을 이용해서 허깅페이스 허브에 로그인합니다.


In [2]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

# [2단계] 데이터셋 불러오기 및 전처리


## 2-1. Hugging Face 데이터셋 불러오기

팀원이 올려둔 KB 금융 데이터셋을 불러옵니다.


In [3]:
from datasets import load_dataset

dataset_name = "sssssungjae/finance-kb-mixed-dataset-final"
hf_ds = load_dataset(dataset_name, split="eval")

print(hf_ds)

README.md:   0%|          | 0.00/444 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/24.0M [00:00<?, ?B/s]

data/eval-00000-of-00001.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20034 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1055 [00:00<?, ? examples/s]

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 1055
})


## 2-2. Qwen 형식 파싱 (question / ground_truth 추출)

데이터셋은 Qwen 학습 포맷(`<|im_start|>user ... <|im_end|>`, `<|im_start|>assistant ... <|im_end|>`)으로 되어 있습니다.  
이를 ragas 평가에 맞게 `question`과 `ground_truth`로 분리합니다.


In [4]:
def parse_qwen_text(row):
    text = row["text"]
    try:
        user_part = text.split("<|im_start|>user")[1].split("<|im_end|>")[0].strip()
        assistant_part = text.split("<|im_start|>assistant")[1].split("<|im_end|>")[0].strip()
    except:
        user_part, assistant_part = None, None
    return {"question": user_part, "ground_truth": assistant_part}

parsed_dataset = hf_ds.map(parse_qwen_text)

# contexts는 없으므로 ragas 형식에 맞춰 더미 값 추가
parsed_dataset = parsed_dataset.add_column("contexts", [[""]] * len(parsed_dataset))

parsed_dataset = parsed_dataset.remove_columns(["text"])
print(parsed_dataset[0])


Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

{'__index_level_0__': 7365, 'question': '적자성 국가채무는 어떤 상황에서 발생하며, 국민에게 어떤 영향을 미칠 수 있는지 설명하시오.', 'ground_truth': '<think>\n\n</think>\n\n적자성 국가채무는 일반적으로 정부의 재정지출이 재정수입을 초과할 때 발생합니다. 이는 다양한 요인에서 비롯될 수 있으며, 이러한 적자는 정부가 자금을 조달하기 위해 채무를 발행하게 만듭니다. 적자성 국가채무가 발생하는 주요 상황과 국민에게 미치는 영향은 다음과 같습니다.\n\n### 적자성 국가채무가 발생하는 상황\n\n1. **경제 침체**: 경기 침체나 경제 불황 시, 세수는 감소하고 동시에 경기 부양을 위한 정부지출이 증가하게 됩니다. 따라서 이러한 상황에서는 적자가 발생할 수 있습니다.\n\n2. **재정 긴축**: 정부가 장기적인 인프라 프로젝트나 복지 지출 확대와 같은 큰 규모의 지출을 계획하면서 즉각적인 수입 증가가 이루어지지 않을 때 적자가 발생할 수 있습니다.\n\n3. **비상 상황**: 전쟁, 자연재해, 팬데믹 등 급작스럽고 큰 비용이 드는 상황에서 정부는 긴급자금을 조달하기 위해 부채를 증가시킬 수밖에 없습니다.\n\n4. **세수 구조의 문제**: 세금 구조가 불안정하거나 경제성장이 둔화되어 세금 수입이 줄어든 경우에 적자가 발생할 수 있습니다.\n\n### 국민에게 미치는 영향\n\n1. **세부담 증가**: 국가채무가 증가하면 미래에 이 부채를 상환하기 위해 세금이 인상될 가능성이 높습니다. 이는 국민의 가처분 소득을 줄이고 경제에 부담을 줄 수 있습니다.\n\n2. **이자 비용 증가**: 국가채무가 많아질수록 정부는 채무에 대한 이자 비용을 감당해야 합니다. 이는 다른 중요한 공공 서비스나 투자에 투입될 수 있는 자금을 줄이게 됩니다.\n\n3. **경제 성장 둔화**: 지속적인 적자는 정부의 재정 여력을 좁히고, 장기적으로 경제 성장에 부정적인 영향을 미칠 수 있습니다. 이는 자본 시장을 불

# [3단계] 평가 대상 모델 로딩


## 3-1. 파인튜닝된 금융 SLM 로딩

Hugging Face Hub에 업로드된 파인튜닝 모델을 불러옵니다.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

my_model_id = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    my_model_id,
    device_map="auto",
    dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

print("모델 로딩 완료 ✅")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

모델 로딩 완료 ✅


# [4단계] 모델 답변 생성



## 4-1. 질문에 대해 모델이 새 답변 생성

`question`을 입력으로 넣고, 파인튜닝 모델이 답변을 생성합니다.

In [8]:
from tqdm import tqdm

# 토크나이저 padding_side 수정 (decoder-only 모델 권장 설정)
tokenizer.padding_side = "left"

questions = parsed_dataset["question"]
contexts = parsed_dataset["contexts"]
ground_truths = parsed_dataset["ground_truth"]

generated_answers = []

batch_size = 8         # T4에서는 8~16 권장
max_new_tokens = 128   # 평가 목적이면 128 정도면 충분

print("\n--- 우리 모델의 답변 생성 (Batch 모드) 시작 ---")
for i in tqdm(range(0, len(questions), batch_size)):
    batch_questions = questions[i:i+batch_size]
    batch_contexts = contexts[i:i+batch_size]

    # 배치 프롬프트 구성
    batch_prompts = [
        f"""Use the following CONTEXT to answer the QUESTION.
        CONTEXT:
        {"\n".join(ctx if isinstance(ctx, list) else [ctx])}
        QUESTION:
        {q}
        """
        for q, ctx in zip(batch_questions, batch_contexts)
    ]

    # chat 템플릿 적용
    chat_prompts = [
        tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
        for prompt in batch_prompts
    ]

    # 배치 인코딩
    inputs = tokenizer(chat_prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # 배치 생성
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

    # 배치 디코딩
    answers = tokenizer.batch_decode(
        outputs[:, inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )
    generated_answers.extend(answers)

print("답변 생성 완료 ✅")



--- 우리 모델의 답변 생성 (Batch 모드) 시작 ---


100%|██████████| 132/132 [16:52<00:00,  7.67s/it]

답변 생성 완료 ✅


# [5단계] Ragas 평가


## 5-1. ragas용 데이터셋 구성

질문(question), 답변(answer), 문맥(contexts), 정답(ground_truth)으로 최종 평가 데이터셋을 구성합니다.

In [1]:
from datasets import Dataset

final_eval_data = {
    "question": questions,
    "answer": generated_answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}
final_eval_dataset = Dataset.from_dict(final_eval_data)


NameError: name 'questions' is not defined

## 5-2. ragas 평가 실행

faithfulness, answer_relevancy, context_precision, context_recall 4가지 지표를 평가합니다.


In [2]:
import os
from google.colab import userdata

# Colab 보안 비밀에 저장된 'OPENAI_API_KEY'를 가져와서 환경 변수로 설정
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("OpenAI API 키 설정이 완료되었습니다.")

OpenAI API 키 설정이 완료되었습니다.


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_openai import ChatOpenAI

evaluator_llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1)


result = evaluate(
    dataset=final_eval_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    llm=evaluator_llm,
)

# 상세 결과
result_df = result.to_pandas()
display(result_df.head())

# 평균 점수
summary = {k: round(v, 3) for k, v in result.items()}
print("\n평균 점수:", summary)